https://en.wikipedia.org/wiki/Zebra_Puzzle

Solution to this, implemented with SymPy

In [46]:
from sympy import symbols, Eq
from sympy.logic.boolalg import Or, And, Xor, Not
from sympy.logic.inference import satisfiable
import re
import pandas as pd

# Create symbols for each value attribute
colours_s = blue, green, ivory, red, yellow = symbols('blue green ivory red yellow')
nationalities_s = Englishman, Japanese, Norwegian, Spaniard, Ukrainian = symbols('Englishman Japanese Norwegian Spaniard Ukrainian')
pets_s = dog, fox, horse, snails, zebra = symbols('dog fox horse snails zebra')
drinks_s = coffee, juice, milk, tea, water = symbols('coffee juice milk tea water')
cigarettes_s = Chesterfields, Kools, LuckyStrike, OldGold, Parliaments = symbols('Chesterfields Kools LuckyStrike OldGold Parliaments')

# Create variables for each entity
n_houses = 5
house_colours = symbols(f'house_colour:{n_houses}')
house_nationalities = symbols(f'house_nationality:{n_houses}')
house_pets = symbols(f'house_pet:{n_houses}')
house_drinks = symbols(f'house_drink:{n_houses}')
house_cigarettes = symbols(f'house_cigarette:{n_houses}')

def Distinct(variables, values):
    constraints = []
    for i in range(len(variables)):
        for j in range(i+1, len(variables)):
            for value in values:
                constraints.append(Not(And(Eq(variables[i], value), Eq(variables[j], value))))
    return And(*constraints)

# Add constraints to ensure each symbol in each category is used only once
distinct_constraints = []
for i in range(n_houses):
    distinct_constraints.append(Xor(*[Eq(house_colours[i], colour) for colour in colours_s]))
distinct_constraints.append(Distinct(house_colours, colours_s))

for i in range(n_houses):
    distinct_constraints.append(Xor(*[Eq(house_nationalities[i], nat) for nat in nationalities_s]))
distinct_constraints.append(Distinct(house_nationalities, nationalities_s))

for i in range(n_houses):
    distinct_constraints.append(Xor(*[Eq(house_pets[i], pet) for pet in pets_s]))
distinct_constraints.append(Distinct(house_pets, pets_s))

for i in range(n_houses):
    distinct_constraints.append(Xor(*[Eq(house_drinks[i], drink) for drink in drinks_s]))
distinct_constraints.append(Distinct(house_drinks, drinks_s))

for i in range(n_houses):
    distinct_constraints.append(Xor(*[Eq(house_cigarettes[i], cig) for cig in cigarettes_s]))
distinct_constraints.append(Distinct(house_cigarettes, cigarettes_s))

# Individual constraints
constraints = []

# The Englishman lives in the red house
constraints.append(Xor(*[And(Eq(house_nationalities[i], Englishman), Eq(house_colours[i], red)) for i in range(n_houses)]))

# The Spaniard owns the dog
constraints.append(Xor(*[And(Eq(house_nationalities[i], Spaniard), Eq(house_pets[i], dog)) for i in range(n_houses)]))

# Coffee is the drink in the green house
constraints.append(Xor(*[And(Eq(house_drinks[i], coffee), Eq(house_colours[i], green)) for i in range(n_houses)]))

# The Ukrainian drinks tea
constraints.append(Xor(*[And(Eq(house_nationalities[i], Ukrainian), Eq(house_drinks[i], tea)) for i in range(n_houses)]))

# The Old Gold smoker owns snails
constraints.append(Xor(*[And(Eq(house_cigarettes[i], OldGold), Eq(house_pets[i], snails)) for i in range(n_houses)]))

# Kools are smoked in the yellow house
constraints.append(Xor(*[And(Eq(house_cigarettes[i], Kools), Eq(house_colours[i], yellow)) for i in range(n_houses)]))

# Milk is the drink in the middle house
constraints.append(Eq(house_drinks[2], milk))

# The Norwegian lives in the first house
constraints.append(Eq(house_nationalities[0], Norwegian))

# The Lucky Strike smoker drinks orange juice
constraints.append(Xor(*[And(Eq(house_cigarettes[i], LuckyStrike), Eq(house_drinks[i], juice)) for i in range(n_houses)]))

# The Japanese smokes Parliaments
constraints.append(Xor(*[And(Eq(house_nationalities[i], Japanese), Eq(house_cigarettes[i], Parliaments)) for i in range(n_houses)]))

# The green house is immediately to the right of the ivory house
constraints.append(Or(*[And(Eq(house_colours[i], ivory), Eq(house_colours[i+1], green)) for i in range(n_houses-1)]))

# The man who smokes Chesterfields lives in the house next to the man with the fox
constraints.append(Or(*[(Eq(house_cigarettes[i], Chesterfields) & Eq(house_pets[i+1], fox)) 
                      | (Eq(house_cigarettes[i+1], Chesterfields) & Eq(house_pets[i], fox)) 
                      for i in range(n_houses-1)]))

# Kools are smoked in the house next to the house where the horse is kept
constraints.append(Or(*[And(Eq(house_cigarettes[i], Kools), Eq(house_pets[i+1], horse)) 
                      | And(Eq(house_cigarettes[i+1], Kools), Eq(house_pets[i], horse)) 
                      for i in range(n_houses-1)]))

# The Norwegian lives next to the blue house
constraints.append(Or(*[And(Eq(house_nationalities[i], Norwegian), Eq(house_colours[i+1], blue)) 
                      | And(Eq(house_nationalities[i+1], Norwegian), Eq(house_colours[i], blue))
                      for i in range(n_houses-1)]))

all_constraints = constraints + distinct_constraints

# Solve the puzzle
solutions = satisfiable(And(*all_constraints), all_models=True)

def print_result(result):
    houses = {}
    # Process each entry in the SymPy result
    for attr, value in [(k.lhs, k.rhs) for k, v in result.items() if v]:
        # Extract the attribute type and house number from the symbol name
        attr_name = str(attr)
        if "nationality" in attr_name:
            attr_type = "nationality"
        elif "drink" in attr_name:
            attr_type = "drink"
        elif "colour" in attr_name:
            attr_type = "colour"
        elif "pet" in attr_name:
            attr_type = "pet"
        elif "cigarette" in attr_name:
            attr_type = "cigarette"
            
        house_id = int(re.search(r"(\d+)$", attr_name).group(1))

        # Initialize the house entry if not already present
        if house_id not in houses:
            houses[house_id] = {"nationality": None, "drink": None, "colour": None, "pet": None, "cigarette": None}

        # Assign the value to the corresponding attribute
        houses[house_id][attr_type] = value

    sorted_houses = {k: houses[k] for k in sorted(houses)}
    display(pd.DataFrame(sorted_houses))    

if solutions:
    print("Solutions:")
    while True:
        try:
            print_result(next(solutions))
        except StopIteration:
            print("No more solutions found.")
            break
else: 
    print("No solution found.")



Solutions:


,0,1,2,3,4
nationality,Norwegian,Ukrainian,Englishman,Spaniard,Japanese
drink,water,tea,milk,juice,coffee
colour,yellow,blue,red,ivory,green
pet,fox,horse,snails,dog,zebra
cigarette,Kools,Chesterfields,OldGold,LuckyStrike,Parliaments


No more solutions found.
